# Freeway

This is the second project for the MC935A/MO436A - Reinforcement Learning course, taught by Prof. Esther Colombini.

In this project we propose to apply Reinforcement Learning methods to teach an agent how to play the Freeway Atari game.

**Group members:**
- Aline Gabriel de Almeida
- Dionisius Oliveira Mayr (229060)
- Marianna de Pinho Severo (264960)
- Victor Jesús Sotelo Chico (265173)

## Freeway game

![Baseline 1](./img/Freeway_logo.png)

Freeway is a video game written by David Crane for the Atari 2600 and published by Activision [[1]](https://en.wikipedia.org/wiki/Freeway_(video_game)).

In the game, two players compete against each other trying to make their chickens cross the street, while evading the cars passing by.
There are three possible actions: staying still, moving forward or moving backward.
Each time a chicken collides with a car, it is forced back some spaces and takes a while until the chicken regains its control.

When a chicken is successfully guided across the freeway, it is awarded one point and moved to the initial space, where it will try to cross the street again.
The game offers multiple scenarios with different vehicles configurations (varying the type, frequency and speed of them) and plays for 2 minutes and 16 seconds.
During the 8 last seconds the scores will start blinking to indicate that the game is close to end.
Whoever has the most points after this, wins the game!

The image was extracted from the [manual of the game](https://www.gamesdatabase.org/Media/SYSTEM/Atari_2600/Manual/formated/Freeway_-_1981_-_Zellers.pdf).

[1 - Wikipedia - Freeway](https://en.wikipedia.org/wiki/Freeway_(video_game))

# Environment

We will be using the [OpenAI Gym](https://gym.openai.com/) toolkit.
This toolkit uses the [Arcade Learning Environment](https://github.com/mgbellemare/Arcade-Learning-Environment) to simulate the game through the [Stella](https://stella-emu.github.io/) emulator.

Although the game offers multiple scenarios, we are going to consider only the first one. Also, we will be controlling a *single chicken*, while we try to maximize its score.

In this configuration, there are ten lanes and each lane contains exactly one car (with a different speed and direction).
Whenever an action is chosen, it is repeated for $k$ frames, $k \in \{2, 3, 4\}$.

This means that our environment is **stochastic** and it is also **episodic**, with its terminal state being reached whenever 2 minutes and 16 seconds have passed.

## Stable Baselines Library

We will be using the [Stable Baselines](https://stable-baselines.readthedocs.io/en/master/) implementations of the Deep Reinforcement algorithms.

It is a fork of the OpenAI Baselines, fully compatible with the OpenAI Gym environments with straightforward usage.

Under the hood, it is using the Tensorflow library to implement the neural networks.

# Setup

Install the dependencies:
```bash
pip install -r requirements.txt
pip install stable-baselines[mpi]
pip install tensorflow==1.15.0
```

# Useful Resources

Here you can find a list of useful links and materials that were used during this project.

* [Freeway-ram-v0 from OpenAI Gym](https://gym.openai.com/envs/Freeway-ram-v0/)
* [Manual of the game](https://www.gamesdatabase.org/Media/SYSTEM/Atari_2600/Manual/formated/Freeway_-_1981_-_Zellers.pdf)
* [Freeway Benchmarks](https://paperswithcode.com/sota/atari-games-on-atari-2600-freeway)

# Imports

In [ ]:
import tensorflow as tf
import gym

from stable_baselines.common import make_vec_env
from stable_baselines.common.cmd_util import make_atari_env
from stable_baselines.common.vec_env import VecFrameStack
from stable_baselines.common.atari_wrappers import make_atari
from stable_baselines.common.policies import MlpPolicy, CnnPolicy

from stable_baselines.deepq.policies import CnnPolicy

from stable_baselines import DQN
from stable_baselines import PPO2

# Action space

As we said above, the agent in this game has three possible actions at each frame, each represented by an integer:

* 0: Stay
* 1: Move forward
* 2: Move backward

In theory, a perfect chicken wouldn't ever need to move backward, since it is possible to know if moving forward would lead you into a collision (in the immediate frame or in the future frames).

# Baseline

## State of the art benchmarks

The image bellow (extracted from https://paperswithcode.com/sota/atari-games-on-atari-2600-freeway) shows the evolution of the scores over time using different techniques.

Today, the state of the art approaches are making 34.0 points, using Deep Reinforcement Learning methods.

![Benchmarks](./img/state_of_art_scores.png)

# Previous Work

On the previous assignment, we explored how classical Reinforcement Learning tabular methods performed on this game.

In some runs, we were able to achieve a peak score of 34 points in a single run using the SARSA($\lambda$) algorithm, which is the state-of-the-art score for this game; but on average we were scoring about 31 points.

# Reward Policy

In the base environment we are awarded on point each time we successfully cross the freeway.

# Methodology

There are a lot of parameters to be tuned in our algorithms.
However, we don't have the required computational power to properly optimize them all.

Since we still want to at least experiment them to observe their impact on the obtained reward, we will be training them for 400k iterations, and then we will select a few of them to train for more iterations.

We are aware that this is far from perfect in the optimization point of view, but this is the best we can do with our available machines.

## Algorithms

We will be comparing two different algorithms here: DQN and PPO.

The idea is to compare an off-policy method (DQN) and an on-policy method (PPO) regarding their stability and number of samples to convergence.

## Tensorboard

Tensorboard allows us to visualize and compare the evolution of the algorithms being tested.
It shows the metrics like the reward over time, loss and advantage, in an easy way.

We will be using it to present our results, understand the impact of different parameters and compare our solutions.

## Frame Stack

Before diving into the experiments' results, it is worth understanding some nuances of our testing environment.

We will be using image representations (frames) of the game as our state.
But by doing so, we end up losing some relavant information of the problem, like the direction the cars are moving.
In other words, the problem we are trying to solve becomes non-markovian.

To solve this, we use the `VecFrameStack` function, that stacks $n$ frames of the game, making our environment become markovian once again.

## Vectorized Environments

[Vectorized environments](https://stable-baselines.readthedocs.io/en/master/guide/vec_envs.html) allows us to stack multiple environments, thus permiting us to traing an agent in $n$ environments per step.

It is controlled by the `num_env` parameter in the `make_atari_env` function.

Although it isn't possible to use vectorized environments with DQN, we will be using it for the PPO2 algorithm.

# Experiments

## DQN

The [Deep-Q-Network](https://arxiv.org/pdf/1312.5602.pdf) is a deep learning model that learns to control policies directly from high dimensional sensory using reinforcement learning.   

The model is a convolutional neural network, trained with a variant of Q-learning, whose input is raw pixels and whose output is a value function estimating the future rewards.  

The Deep-Q-Network algorithm observes the image $x_t$ from the emulator which is a vector of raw pixel values representing the current screen. In addition it receives a reward $r_t$ representing the change in game score.  

It considers sequences of actions and observations,  

$s_t = x_1, a_1, x_2, ... a_{t-1}x_t$,  

and learn game strategies that depend upon these sequences.  


The optimal action-value function obeys an important identity known as the Bellman equation. This is based on the following intuition: if the optimal value $Q*(s', a')$ of the sequence $s'$ at the next time-step was known for all possible actions $a'$, then the optimal strategy is to select the action $a'$
maximising the expected value of $r + \gamma Q*(s', a')$, where $\gamma$ is the reward discount factor per time-step,  
  
$Q*(s, a) = E_{s' ~ \epsilon}[r + \gamma max_{a'}Q*(s', a')|s, a]$  

     


In this project we applied the [algorithm implemented by Stable Baselines](https://stable-baselines.readthedocs.io/en/master/modules/dqn.html) to the Atari Freeway game.

### Influence of the discount factor $\gamma$

The discount factor $\gamma$ determines how much the agent cares about rewards in the distant future relative to those in the immediate future.  
  
If $\gamma$=0, the agent will be completelly myopic and only learn about actions that produce an immediate reward.  

If $\gamma$=1, the agent will evaluate each of its actions based on the sum of total of all futures rewards.  
  
We used a $\gamma$ value of 0.99 in order to make our agent care about distant future and we also decreased this value to 0.90 and 0.75 to see how they can impact the agent behavior.  

Thus, we will be experimenting with 3 different parameters set:

| Parameter | G1 | G2 | G3 |
|------|----|----|----|
| **`GAMMA`** | 0.99 | 0.90 | 0.75 |
| `LEARNING_RATE` | 0.0005 | 0.0005 | 0.0005 |
| `EXPLORATION_RATE` | 0.1 | 0.1 | 0.1 |
|`Smoothed Reward` |20.73|23.25|21.72|


<center><img src=https://raw.githubusercontent.com/DionisiusMayr/FreewayGame/main/aline.almeida/DQN/plots/dqn_gamma.png width="400"></center>

| $\gamma$=0.99 | $\gamma$=0.90 | $\gamma$=0.75 |  
|---|---|---|  
| <img src=https://raw.githubusercontent.com/DionisiusMayr/FreewayGame/main/aline.almeida/DQN/plots/vermelho.png width="250"> | <img src =https://raw.githubusercontent.com/DionisiusMayr/FreewayGame/main/aline.almeida/DQN/plots/pink.png width="250"> | <img src=https://raw.githubusercontent.com/DionisiusMayr/FreewayGame/main/aline.almeida/DQN/plots/azul_claro.png width="250"> |

## Learning Rate


We will be experimenting with 3 different parameters set:

| Parameter | G1 | G2 | G3 |
|------|----|----|----|
| `GAMMA` | 0.99 | 0.99 | 0.99 |
| **`LEARNING_RATE`** | 0.0005 | 0.0010 | 0.0050 |
| `EXPLORATION_RATE` | 0.1 | 0.1 | 0.1 |
|`Smoothed Reward` |20.73|21.13|2.616e-19 (approx. 0)|

<center><img src=https://raw.githubusercontent.com/DionisiusMayr/FreewayGame/main/aline.almeida/DQN/plots/dqn_lr.png width="400"></center>

| `LEARNING_RATE`=0.0005 | `LEARNING_RATE`=0.0010 | `LEARNING_RATE`=0.0050 |  
|---|---|---|  
| <img src=https://raw.githubusercontent.com/DionisiusMayr/FreewayGame/main/aline.almeida/DQN/plots/vermelho.png width="250"> | <img src=https://raw.githubusercontent.com/DionisiusMayr/FreewayGame/main/aline.almeida/DQN/plots/cinza.png width="250"> | <img src=https://raw.githubusercontent.com/DionisiusMayr/FreewayGame/main/aline.almeida/DQN/plots/verde.png width="250"> |

### Influence of the agent's exploration rate

The exploration rate is the probability that our agent will explore the environment rather than exploit it.  

We used 0.1 as our baseline exploration value. In order to see how the exploration rate impact the agent behavior, we also made experiments using the double of this value (0.1) and the half of it (0.05).

All in all, these are the parameters that we are going to use to execute this experiment:

| Parameter | G1 | G2 | G3 |
|------|----|----|----|
| `GAMMA` | 0.99 | 0.99 | 0.99 |
| `LEARNING_RATE` | 0.0005 | 0.0005 | 0.0005 |
| **`EXPLORATION_RATE`** | 0.1 | 0.05 | 0.20 |
|`Smoothed Reward` |20.73|22.02|21.48|

<center><img src=https://raw.githubusercontent.com/DionisiusMayr/FreewayGame/main/aline.almeida/DQN/plots/dqn_exploration.png witdh="400"></center>

| `EXPLORATION_RATE`=0.0020 | `EXPLORATION_RATE`=0.0010 | `EXPLORATION_RATE`=0.0005 |  
|---|---|---|  
| <img src=https://raw.githubusercontent.com/DionisiusMayr/FreewayGame/main/aline.almeida/DQN/plots/laranja.png width="250"> | <img src=https://raw.githubusercontent.com/DionisiusMayr/FreewayGame/main/aline.almeida/DQN/plots/vermelho.png width="250"> | <img src=https://raw.githubusercontent.com/DionisiusMayr/FreewayGame/main/aline.almeida/DQN/plots/azul.png width="250"> |

## PPO2

In [5]:
TOTAL_TIMESTEPS = 400000

In [38]:
def experiment(tb_lob, gamma, learning_rate, lam, policy, timesteps=TOTAL_TIMESTEPS):
    # multiprocess environment
    env = make_atari_env('FreewayNoFrameskip-v0', num_env=8, seed=0)
    # Frame-stacking with 4 frames
    env = VecFrameStack(env, n_stack=4)

    model = PPO2(policy, env, verbose=0, tensorboard_log=tb_lob, gamma=gamma, learning_rate=learning_rate, lam=lam)
    model.learn(total_timesteps=timesteps)
    return model

## Learning Rate

In [25]:
GAMMA = 0.99
LEARNING_RATE = 0.00025
LAM = 0.95

In [26]:
experiment('Experiments_PPO_A', gamma=GAMMA, learning_rate=LEARNING_RATE, lam=LAM, policy=MlpPolicy);

In [28]:
experiment('Experiments_PPO_A', gamma=GAMMA, learning_rate=0.00025 * 0.25, lam=LAM, policy=MlpPolicy);

In [30]:
experiment('Experiments_PPO_A', gamma=GAMMA, learning_rate=0.00025 * 0.5, lam=LAM, policy=MlpPolicy);

In [32]:
experiment('Experiments_PPO_A', gamma=GAMMA, learning_rate=0.00025 * 2, lam=LAM, policy=MlpPolicy);

In [34]:
experiment('Experiments_PPO_A', gamma=GAMMA, learning_rate=0.00025 * 4, lam=LAM, policy=MlpPolicy);

|Parameter|A1|A2|A3|A4|A5|
|-|-|-|-|-|-|
|GAMMA|0.99|0.99|0.99|0.99|0.99|
|LEARNING_RATE|$$2.5 \cdot 10^{-4}$$|$$6.25 \cdot 10^{-5}$$|$$1.25 \cdot 10^{-4}$$|$$5 \cdot 10^{-4}$$|$$1 \cdot 10^{-3}$$|
|LAM|0.95|0.95|0.95|0.95|0.95|
|Smoothed Reward|21.49|21.41|0.0|21.21|21.17|

|![](./img/alpha_complete.png)|
|-| 

|LEARNING_RATE = $$2.5 \cdot 10^{-4}$$|LEARNING_RATE = $$6.25 \cdot 10^{-5}$$|LEARNING_RATE = $$1.25 \cdot 10^{-4}$$|
|-|-|-|
|![](./img/alpha_2.5-4.png)|![](./img/alpha_6.25-5.png)|![](./img/alpha_1.25-4.png)|

|LEARNING_RATE = $$5 \cdot 10^{-4}$$|LEARNING_RATE = $$1 \cdot 10^{-3}$$|
|-|-|
|![](./img/alpha_5-4.png)|![](./img/alpha_1-3.png)|

## LAM

The `lam` is a factor that controls the trade-off of bias vs variance for Generalized Advantage Estimator [(PPO2 doc)](https://stable-baselines.readthedocs.io/en/master/modules/ppo2.html).

In [7]:
GAMMA = 0.99
LEARNING_RATE = 0.00025
LAM = 0.95

In [8]:
experiment('Experiments_PPO_L', gamma=GAMMA, learning_rate=LEARNING_RATE, lam=LAM, policy=MlpPolicy);






Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where







In [10]:
experiment('Experiments_PPO_L', gamma=GAMMA, learning_rate=LEARNING_RATE, lam=1.00, policy=MlpPolicy);

In [12]:
experiment('Experiments_PPO_L', gamma=GAMMA, learning_rate=LEARNING_RATE, lam=0.99, policy=MlpPolicy);

In [14]:
experiment('Experiments_PPO_L', gamma=GAMMA, learning_rate=LEARNING_RATE, lam=0.5, policy=MlpPolicy);

In [16]:
experiment('Experiments_PPO_L', gamma=GAMMA, learning_rate=LEARNING_RATE, lam=0.25, policy=MlpPolicy);

In [18]:
experiment('Experiments_PPO_L', gamma=GAMMA, learning_rate=LEARNING_RATE, lam=0.0, policy=MlpPolicy);

|Parameter|L1|L2|L3|L4|L5|L6|
|-|-|-|-|-|-|-|
|GAMMA|0.99|0.99|0.99|0.99|0.99|0.99|
|LEARNING_RATE|0.00025|0.00025|0.00025|0.00025|0.00025|0.00025|
|LAM|0.95|1.00|0.99|0.5|0.25|0.0|
|Smoothed Reward|21.62|23.71|22.51|21.88|24.28|20.39|

|![](./img/lam_complete.png)|
|-|

|LAM = 0.95|LAM = 1.00|LAM = 0.99|
|-|-|-|
|![](./img/lam_0.95.png)|![](./img/lam_1.0.png)|![](./img/lam_0.99.png)|

|LAM = 0.5|LAM = 0.25|LAM = 0.0|
|-|-|-|
|![](./img/lam_0.5.png)|![](./img/lam_0.25.png)|![](./img/lam_0.0.png)|

From the graphs above we can see that the LAM parameter doesn't seem to have an impactful relationship with the overall performance of the algorithm.

Although the 0.25 lam ended up with a smoothed score of 24.28, it seems to be really unstable, and no apparent relationship with this parameter could be noted.

## Variability of experiments

Here we will be exploring how unstable a run of the algorithm is by repeating it multiple times with the same parameter settings.

In [20]:
GAMMA = 0.99
LEARNING_RATE = 0.00025
LAM = 0.95

In [21]:
experiment('Experiments_PPO_Var', gamma=GAMMA, learning_rate=LEARNING_RATE, lam=LAM, policy=MlpPolicy);

In [22]:
experiment('Experiments_PPO_Var', gamma=GAMMA, learning_rate=LEARNING_RATE, lam=LAM, policy=MlpPolicy);

In [ ]:
experiment('Experiments_PPO_Var', gamma=GAMMA, learning_rate=LEARNING_RATE, lam=LAM, policy=MlpPolicy);

|Parameter|R1|R2|R3|
|-|-|-|-|
|GAMMA|0.99|0.99|0.99|
|LEARNING_RATE|0.00025|0.00025|0.00025|
|LAM|0.95|0.95|0.95|
|Smoothed Reward|22.68|21.75|21.75|

|![](./img/var_complete.png)|
|-|

|R1|R2|R3|
|-|-|-|
|![](./img/var_1.png)|![](./img/var_2.png)|![](./img/var_3.png)|

---

# Final Thoughts ------------- TODO TODO TODO

## Computational cost

One of the biggest problems on this project was the computational cost of running an episode.

Since each run plays for 2 minutes and 16 seconds in the original game, there are quite a lot of frames that need to be computed for each episode.
Even though the frame-sync is deactivated in our environment, each time we execute on episode it takes about 2 seconds to compute it for Q-Learn and Monte Carlo, but it takes around 21s seconds for SARSA($\lambda$)! This means that in order to run 4k episodes, one must wait 23 hours. And since we want to experiment with 6 different values of $\lambda$, it would take about a week to compute this all using a single core processor.

The memory usage is fairly low compared to the time that it takes to run the algorithms, even with a decent amount of unique states in our problem.

## Optimality

Regarding the optimality of our solution, we were able to achive the state-of-the-art score (34.0) some times with the SARSA($\lambda$) algorithm, but on average we are closer to 31 points (which is also good, having said that our baseline was 21.8.
Q-Learning also showed really good results, achieving about 28 points on average.
On the other hand, Monte Carlo didn't perform well in our problem, achieving only 13 points on average.

## Regarding the Linear Function Approximators

According to the experiments realized with function approximation for Monte Carlo, Q Learning and Sarsa Lambda control algorithms, we achieved some meaningful results: 

* We were not able to improve the agents compared to the algorithms without function approximation. Some factors may have contributed to this, such as the simplicity of the linear approximator for a problem that, probably, have many nonlinear relationships between the environment variables. Also, the features that we created may not be good enough to provide meaningful information about the environment to the agents.

* The feature approximation of Monte Carlo said that we have to take a sample for our model in order to update the weight values. However, for our problems, this sample is the entire game, so each new generated sample follows a past policy that makes the convergence slow.

* Linear approximators can be faster than its original counterpart as we saw with SARSA, but it’s not always the case.

* To add randomness to underfitted models by using a bigger N0 can be beneficial to the solution.

* Each algorithm has its own specificities, that depends on how it works. For example, for the Monte Carlo approximator, using a less sparse feature vector contributed for better results. On the other hand, for the Q-Learning and Sarsa Lambda approximators, providing a bigger exploration capacity for the agent was very important to achieve better results.

* In all the experiments, the agents converged to a behavior similar to that of the baseline, learning that moving up the most part of the time is the best policy. However, as we saw from the algorithms without approximation, that is not the truth.

All in all, we were satisfied with what we achieved, being able to apply multiple tabular methods and experiment a lot on Reinforcement Learning.